In [1]:
import requests

url = "https://data.sec.gov/submissions/CIK0000320193.json"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Send a GET request to the URL with headers
response = requests.get(url, headers=headers)

if response.status_code == 200:
    json_data = response.json()
    # print(json_data)
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")

In [2]:
filings = json_data['filings']

In [3]:
json_data.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [14]:
len(filings['recent']['accessionNumber'])

1000

In [15]:
len(filings['recent']['form'])

1000

In [16]:
filings['recent'].keys()

dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])

In [7]:
# filings['recent']['filingDate']

In [7]:
for i,j in zip(filings['recent']['accessionNumber'],filings['recent']['form']):
    if j == '10-K': 
        print(i)
        break

0000320193-23-000106


In [13]:
# filings['recent']['form']

In [6]:
from prepline_sec_filings.fetch import (
    get_form_by_ticker, open_form_by_ticker, get_filing
)

# Fetch the 2021 10-K instead of the most recent one
rgld_cik = 320193
accession_number = "000032019323000106"
text = get_filing(rgld_cik,
                  accession_number,
                  company='Unstructured Technologies', 
                  email='support@unstructured.io')

In [11]:
print(text[1375:3284])

d04-4f75-a78c-e87ea3423788,d:1cb1ba018cb1455aa66bd3f9ab0c5b1a--><html xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:aapl="http://www.apple.com/20230930" xmlns="http://www.w3.org/1999/xhtml" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:srt="http://fasb.org/srt/2023" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:country="http://xbrl.sec.gov/country/2023" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xml:lang="en-US"><head><meta http-equiv="Content-Type" content="text/html"/>


<title>aapl-20230930</title></head><body><div style="display:none"><ix:header><ix:hidd

In [12]:

# pipeline-api
from prepline_sec_filings.sections import section_string_to_enum, validate_section_names, SECSection
from prepline_sec_filings.sec_document import SECDocument, REPORT_TYPES, VALID_FILING_TYPES

sec_document = SECDocument.from_string(text)
risk_narrative = sec_document.get_section_narrative(SECSection.RISK_FACTORS)

for element in risk_narrative[:3]:
    print(element)
    print("\n")


The Company’s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize from time to time, the Company’s business, reputation, results of operations, financial condition and stock price can be materially and adversely affected.


Because of the following factors, as well as other factors affecting the Company’s results of operations and financial condition, past financial performance should not be considered to be a reliable indicator of future performance, and investors should not use historical trends to anticipate results or trends in future periods. This discussion of risk factors contains forward-looking statements.


This section should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial state

In [1]:
from prepline_sec_filings.fetch import get_cik_by_ticker

get_cik_by_ticker('AAPL')

'0000320193'

In [2]:
cik = '0000320193'

10

In [3]:
get_cik_by_ticker('AMZN')

'0001018724'

In [4]:
get_cik_by_ticker('GOOGL')

'0001652044'

In [8]:
s = '0001652044'
int(s.strip('0'))

1652044

In [11]:
from typing import List
import re
from src.sec_filings import SECExtractor
import concurrent.futures
from functools import partial
from src.prepline_sec_filings.fetch import get_cik_by_ticker
import requests
from src.prepline_sec_filings.fetch import (
    get_form_by_ticker, open_form_by_ticker, get_filing
)
def sec_main(ticker:str,year:str,forms:List[str] = ['10-K','10-Q']):
    cik = get_cik_by_ticker(ticker)
    rgld_cik = int(cik.strip('0'))
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send a GET request to the URL with headers
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        json_data = response.json()
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
    
    form_lists = []
    filings = json_data['filings']
    recent_filings = filings['recent']
    quarter_val = 3
    for acc_num,form_name,filing_date,report_date in zip(recent_filings['accessionNumber'],recent_filings['form'],recent_filings['filingDate'],recent_filings['reportDate']):
        if form_name in forms and report_date.startswith(str(year)):
            print(form_name)
            if form_name == "10-Q":
                form_name+=str(quarter_val)
                quarter_val-=1
            no_dashes_acc_num = re.sub("-","",acc_num)
            form_lists.append([no_dashes_acc_num,form_name,filing_date,report_date])
    
    acc_nums_list = [l[0] for l in form_lists]

    get_filing_partial = partial(get_filing,
                                 cik=rgld_cik,
                                company='Unstructured Technologies', 
                                email='support@unstructured.io')
    
    sec_extractor = SECExtractor(ticker=ticker)

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = executor.map(get_filing_partial, acc_nums_list)
    results_texts = []
    for res in results:
        results_texts.append(res)
    assert len(results_texts) == len(acc_nums_list), f"The scraped text {len(results_texts)} is not matching with accession number texts {len(acc_nums_list)}"
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        results = executor.map(sec_extractor.get_section_texts_from_text,results_texts)
    section_texts = []
    for res in results:
        section_texts.append(res)
    assert len(section_texts) == len(acc_nums_list), f"The section text {len(section_texts)} is not matching with accession number texts {len(acc_nums_list)}"

    for idx,val in enumerate(form_lists):
        val.append(section_texts[idx])
    return form_lists
    

In [1]:
from src.secData import sec_main as sm_
%load_ext autoreload
%autoreload 2
texts,fm = sm_('WAT',2023)

[nltk_data] Downloading package punkt to /home/athekunal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[]
['10-Q3']
['10-Q3', '10-Q3-1']
Started Scraping
Scraped
Started Extracting
Extracted


In [2]:
fm

['10-Q3', '10-Q3-1', '10-Q2']

In [21]:
from langchain.schema import Document

docs = []

for filings in appl_texts:
    # print(filings)filingDate
    texts_dict = filings[-1]
    
    for section_name, text in texts_dict.items():
        docs.append(Document(page_content=text,metadata={"accessionNumber":filings[0],"filing_type":filings[1],"filingDate":filings[2],"reportDate":filings[3],'sectionName':section_name}))

In [24]:
print({"hello":1,"there":2})

{'hello': 1, 'there': 2}


In [1]:
f = ["10-K","10-Q"]
g = ["10-K","10-Q", "10-K/A","10-Q/A"]

In [3]:
include_amends = True
k = []
if include_amends:
    for form in f:
        k.append(form)
        k.append(form+"/A")

## QUERYING DATABASE

In [16]:
from src.vectorDatabase import create_database
qdrant_client, encoder, speakers_list_1, speakers_list_2, speakers_list_3 = create_database("AMZN",2023)

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
SEC
Started Scraping
Scraped
Started Extracting


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Extracted


In [17]:
%load_ext autoreload
%autoreload 2
from src.queryDatabase import query_database_earnings_call, query_database_sec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
text = query_database_earnings_call(
    question= "What did Amazon do on generative AI?", quarter="Q3", qdrant_client=qdrant_client, encoder=encoder, speakers_list = speakers_list_3
)

In [19]:
print(text)

Andrew Jassy: United Airlines, to name a few. We are also seeing success with generative AI start-ups like Perplexity.ai who chose to go all in with AWS, including running future models in Trainium and Inferentia. And the AWS team has a lot of new capabilities to share with its customers at its upcoming AWS re:Invent conference. Beyond AWS, all of our significant businesses are working on generative AI applications to transform their customer experiences. There are too many for me to name on this call but a few exampleslike we do, that equation is going to flip. There's a lot more there for us. And then you look at the very substantial gigantic new generative AI opportunity which I believe will be tens of billions of dollars of revenue for AWS over the next several years, I think we have a unique and broad approach that's really resonating with customers. And you can see it with the array of customers I mentioned that are using us and starting to build workloads for generative AI who h

In [20]:
text = query_database_sec(
    question= "What did Apple do on generative AI?",
    qdrant_client=qdrant_client,encoder=encoder,
    search_form="10-Q1"
)

In [21]:
print(text)

LEGAL_PROCEEDINGS: District Court for the Northern District of New York. The complaint alleges, among other things, that “Alexa Voice Software and Alexa enabled devices” infringe U.S. Patent No. 7,177,798, entitled “Natural Language Interface Using Constrained Intermediate Dictionary of Results.” The complaint seeks an injunction, an unspecified amount of damages, enhanced damages, an ongoing royalty, interest, attorneys’ fees, and costs. In March 2023, the plaintiffs alleged in their damages report that in the event of a. The Company is involved from time to time in claims, proceedings, and litigation, including the matters described in Item 8 of Part II, “Financial Statements and Supplementary Data — Note 7 — Commitments and Contingencies — Legal Proceedings” of our 2022 Annual Report on Form 10-K, as supplemented by the following: In May 2018, Rensselaer Polytechnic Institute and CF Dynamic Advances LLC filed a complaint against Amazon.com, Inc. in the United States District Court f

In [22]:
s = "2023"
int(s)

2023

In [35]:
import os
from dotenv import load_dotenv
from functools import lru_cache

@lru_cache
def return_env():
    return os.environ["HELLO"]

def set_env():
    os.environ["HELLO"] = "dsf"
    # return_env(True)
    # load_dotenv(verbose=True, override=True)
# set_env()

def print_env():
    print(os.environ["HELLO"])

In [36]:
if True:
    set_env()
print_env()

dsf


In [37]:
from qdrant_client import QdrantClient

qd = QdrantClient("sec-earnings-call-db")

In [6]:
# import module 
import contextvars 

# declaring the variable 
# to it's default value 
data = ["1"]
cvar = contextvars.ContextVar("cvar", 
							default = data) 

print("value of context variable cvar: \n", 
	cvar.get()) 

# calling set method 
token = cvar.set(["1","2"]) 

print("\nvalue after calling set method: \n", 
	cvar.get()) 

token = cvar.set(["11","2"]) 
print(cvar)
print("\nvalue after calling set method: \n", 
	cvar.get()) 
# checking the type of token instance 
print("\nType of object instance returned by set method: \n", 
	type(token)) 

# calling the reset method. 
cvar.reset(token) 

print("\nvalue after calling reset method: \n", 
	cvar.get())


value of context variable cvar: 
 ['1']

value after calling set method: 
 ['1', '2']
<ContextVar name='cvar' default=['1'] at 0x7f46e5743bf0>

value after calling set method: 
 ['11', '2']

Type of object instance returned by set method: 
 <class '_contextvars.Token'>

value after calling reset method: 
 ['1', '2']


In [1]:
from src.vectorDatabase import get_all_docs

docs,sec_form_names,earnings_call_quarter_vals,speakers_list_1,speakers_list_2,speakers_list_3 = get_all_docs("WAT",2023)

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
Earnings Call Q4
SEC
Started Scraping
Scraped
Started Extracting
Extracted


In [2]:
sec_form_names

['10-Q3', '10-Q3', '10-Q2']

In [4]:
from src.vectorDatabaseDocker import create_database

create_database('META','2024')

Earnings Call Q1
Earnings Call Q2
Don't have the data for Q2
Earnings Call Q3
Don't have the data for Q3
Earnings Call Q4
Don't have the data for Q4
SEC
Started Scraping
Scraped
Started Extracting
Extracted


/home/athekunal/Finance-Data-LLM-Project/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(<qdrant_client.qdrant_client.QdrantClient at 0x7fe468263ac0>,
 SentenceTransformer(
   (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
   (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
   (2): Normalize()
 ),
 ['Kenneth Dorell',
  'Mark Zuckerberg',
  'Susan Li',
  'Operator',
  'Eric Sheridan',
  'Mark Zuckerberg',
  'Operator',
  'Brian Nowak',
  'Susan Li',
  'Operator',
  'Mark Shmulik',
  'Susan Li',
  'Operator',
  'Douglas Anmuth',
  'Mark Zuckerberg',
  'Susan Li',
  'Operator',
  'Justin Post',
  'Susan Li',
  'Mark Zuckerberg',
  'Operator',
  'Youssef Squali',
  'Susan Li',
  'Operator',
  'Ken Gawrelski',
  'Susan Li',
  'Operator',
  'Ross Sandler',
  'Mark Zuckerberg',
  'Kenneth Dorell',
  'Operator'